In [11]:
# env setup
!pip install pandas pyarrow tqdm regex seaborn matplotlib

import os
from pathlib import Path

BASE_DIR = Path(".").resolve()
DATA_DIR = BASE_DIR / "parquet"
RESULTS_DIR = BASE_DIR / "results2"
CONTEXT_DIR = RESULTS_DIR / "contexts2"
LLM_DIR = RESULTS_DIR / "llm_results2"

for d in [RESULTS_DIR, CONTEXT_DIR, LLM_DIR]:
    d.mkdir(exist_ok = True)

print("Directories ready:")
print(BASE_DIR, DATA_DIR, RESULTS_DIR)


[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Directories ready:
/Users/amaanahmed259/Downloads/16919051 /Users/amaanahmed259/Downloads/16919051/parquet /Users/amaanahmed259/Downloads/16919051/results2


In [12]:
# load data
import pandas as pd

tables = {}
for name in [
    "pull_request", "pr_timeline", "pr_reviews", "pr_review_comments",
    "pr_commits", "pr_commit_details", "repository", "pr_comments"
]:
    path = DATA_DIR / f"{name}.parquet"
    if path.exists():
        tables[name] = pd.read_parquet(path)
        print(f"✅ Loaded {name}: {len(tables[name]):,} rows")
    else:
        print(f"⚠️ Missing {name}.parquet")

✅ Loaded pull_request: 33,596 rows
✅ Loaded pr_timeline: 325,500 rows
✅ Loaded pr_reviews: 28,875 rows
✅ Loaded pr_review_comments: 19,450 rows
✅ Loaded pr_commits: 88,576 rows
✅ Loaded pr_commit_details: 711,923 rows
✅ Loaded repository: 2,807 rows
✅ Loaded pr_comments: 39,122 rows


In [ ]:
# compute refined failed_prs (closed_unmerged + stale_open)
import numpy as np
import pandas as pd
from datetime import datetime, timezone

pull = tables["pull_request"].copy()
timeline = tables.get("pr_timeline")
reviews = tables.get("pr_reviews")

# ensure timestamps are proper datetimes for any columns that exist
for col in ["created_at", "updated_at", "closed_at", "merged_at"]:
    if col in pull.columns:
        pull[col] = pd.to_datetime(pull[col], errors = "coerce", utc = True)

# --------------------------------------------------------------------
# Base failure definition: CLOSED and NOT MERGED  => "closed_unmerged"
# --------------------------------------------------------------------

pull["state_lower"] = pull["state"].astype(str).str.lower()
closed_unmerged_mask = (pull["state_lower"] == "closed") & pull["merged_at"].isna()

closed_unmerged = pull[closed_unmerged_mask].copy()
closed_unmerged["outcome"] = "closed_unmerged"

print("Closed & not merged:", len(closed_unmerged))

# --------------------------------------------------------------------
# Stale open PRs => "stale_open"
#    We need some notion of "last activity":
#    - Prefer 'updated_at' if present
#    - Otherwise fall back to 'created_at'
# --------------------------------------------------------------------

now = pd.Timestamp(datetime.now(timezone.utc))
stale_days = 180

open_mask = pull["state_lower"] == "open"

# build a generic "last_activity" timestamp
if "updated_at" in pull.columns:
    pull["last_activity"] = pull["updated_at"].where(pull["updated_at"].notna(), pull["created_at"])
else:
    # no updated_at column, use created_at as a proxy
    pull["last_activity"] = pull["created_at"]

stale_open_mask = (
    open_mask
    & pull["last_activity"].notna()
    & ((now - pull["last_activity"]) > pd.Timedelta(days = stale_days))
    & pull["created_at"].notna()
    & ((now - pull["created_at"]) > pd.Timedelta(days = stale_days))
)

stale_open = pull[stale_open_mask].copy()
stale_open["outcome"] = "stale_open"

print("Stale open (treated as failures):", len(stale_open))

# --------------------------------------------------------------------
# Combine failure sets
# --------------------------------------------------------------------

failed_prs = pd.concat([closed_unmerged, stale_open], ignore_index = True)

# make sure id is int (some operations later assume this)
failed_prs["id"] = failed_prs["id"].astype(int)

print("Total failed PRs:", len(failed_prs))

# --------------------------------------------------------------------
# Add CI failure and CHANGES_REQUESTED indicators AS SIGNALS
# --------------------------------------------------------------------

ci_failed_ids = set()
if timeline is not None and {"event", "pr_id"}.issubset(timeline.columns):
    mask = timeline["event"].astype(str).str.contains("fail|workflow_failed|error", case = False, na = False)
    ci_failed_ids = set(timeline.loc[mask, "pr_id"].astype(int))

cr_ids = set()
if reviews is not None and {"state", "pr_id"}.issubset(reviews.columns):
    mask = reviews["state"].astype(str).str.contains("changes_requested", case = False, na = False)
    cr_ids = set(reviews.loc[mask, "pr_id"].astype(int))

failed_prs["ci_failed"] = failed_prs["id"].isin(ci_failed_ids)
failed_prs["changes_requested"] = failed_prs["id"].isin(cr_ids)

failed_prs.to_csv(RESULTS_DIR / "failed_prs_refined.csv", index = False)
print("Saved:", RESULTS_DIR / "failed_prs_refined.csv")

failed_prs.head()


Closed & not merged: 7270
Stale open (treated as failures): 365
Total failed PRs: 7635
Saved: /Users/amaanahmed259/Downloads/16919051/results2/failed_prs_refined.csv


,id,number,title,body,agent,user_id,user,state,created_at,closed_at,merged_at,repo_id,repo_url,html_url,state_lower,outcome,last_activity,ci_failed,changes_requested
0,3264933329,2911,Fix: Wait for all partitions in load_collectio...,## Summary\n\nFixes an issue where `load_colle...,Claude_Code,108661493,weiliu1031,closed,2025-07-26 02:59:01+00:00,2025-07-29 07:01:20+00:00,NaT,191751505,https://api.github.com/repos/milvus-io/pymilvus,https://github.com/milvus-io/pymilvus/pull/2911,closed,closed_unmerged,NaT,False,False
1,3234102722,318,chore: Convert hive-mind coordination system t...,## Summary\n\nThis PR converts the AI agent co...,Claude_Code,15803865,lanemc,closed,2025-07-16 01:00:34+00:00,2025-07-17 12:49:29+00:00,NaT,995029641,https://api.github.com/repos/ruvnet/claude-flow,https://github.com/ruvnet/claude-flow/pull/318,closed,closed_unmerged,NaT,False,False
2,3212961701,113,fix: Add missing logger.js and resolve MCP ser...,## Summary\n\nFixed critical MCP server issues...,Claude_Code,2934394,ruvnet,closed,2025-07-08 15:26:38+00:00,2025-07-08 15:27:58+00:00,NaT,1009254201,https://api.github.com/repos/ruvnet/ruv-FANN,https://github.com/ruvnet/ruv-FANN/pull/113,closed,closed_unmerged,NaT,False,False
3,3164503419,40,Fix Claude animation flickering with vt10x-ins...,## 🎯 Problem: Claude's Thinking Animation Caus...,Claude_Code,2891702,hjanuschka,closed,2025-06-20 22:47:18+00:00,2025-06-21 11:51:22+00:00,NaT,1002552148,https://api.github.com/repos/amantus-ai/vibetu...,https://github.com/amantus-ai/vibetunnel/pull/40,closed,closed_unmerged,NaT,False,False
4,3165440543,1030,Fix agents page module resolution error and up...,## Summary\n- Fixed critical module resolution...,Claude_Code,14167547,AtlantisPleb,closed,2025-06-21 21:22:44+00:00,2025-06-21 22:53:43+00:00,NaT,715683924,https://api.github.com/repos/OpenAgentsInc/ope...,https://github.com/OpenAgentsInc/openagents/pu...,closed,closed_unmerged,NaT,False,False


In [ ]:
from tqdm import tqdm

def truncate(s, max_chars=1500):
    s = str(s or "")
    return s if len(s) <= max_chars else s[:max_chars//2] + "\n...\n" + s[-max_chars//2:]

def safe_str(x):
    try: return str(x)
    except: return ""

In [ ]:
# build context files for each failed PR (using refined failed_prs)
from tqdm import tqdm

commit_details = tables.get("pr_commit_details")
repos          = tables.get("repository")
timeline       = tables.get("pr_timeline")
reviews        = tables.get("pr_reviews")
rev_comments   = tables.get("pr_review_comments")
pr_comments    = tables.get("pr_comments")
pr_commits     = tables.get("pr_commits")

contexts = []

MAX_CONTEXTS = None  # e.g., 500 for debugging, None for full dataset

if MAX_CONTEXTS is not None:
    iter_df = failed_prs.head(MAX_CONTEXTS)
else:
    iter_df = failed_prs

for _, pr in tqdm(iter_df.iterrows(), total=len(iter_df)):
    pid = int(pr["id"])
    repo_id = pr.get("repo_id", None)

    # repository metadata
    repo_name = "unknown"
    repo_lang = "unknown"
    if repos is not None and not repos.empty and "id" in repos.columns:
        rsub = repos[repos["id"] == repo_id]
        if not rsub.empty:
            rrow = rsub.iloc[0]
            repo_name = safe_str(rrow.get("name", "unknown"))
            repo_lang = safe_str(rrow.get("language", "unknown"))

    # author metadata (AIDev may have author_type)
    author_login = safe_str(pr.get("user_login") or pr.get("author_login"))
    author_type  = safe_str(pr.get("author_type", "unknown"))

    state     = safe_str(pr.get("state"))
    outcome   = safe_str(pr.get("outcome", "closed_unmerged"))
    created_at = safe_str(pr.get("created_at"))
    updated_at = safe_str(pr.get("updated_at"))
    closed_at  = safe_str(pr.get("closed_at"))
    merged_at  = safe_str(pr.get("merged_at"))

    ci_failed_flag         = bool(pr.get("ci_failed", False))
    changes_requested_flag = bool(pr.get("changes_requested", False))

    # -----------------------------
    # Diff summary (from pr_commit_details)
    # -----------------------------
    
    diff_snippet = ""
    if commit_details is not None and "pr_id" in commit_details.columns:
        diff_df = commit_details[commit_details["pr_id"] == pid]
        if not diff_df.empty:
            parts = []
            for _, row in diff_df.head(20).iterrows():
                fname = safe_str(row.get("filename"))
                patch = truncate(row.get("patch", ""), 400)
                parts.append(f"File: {fname}\n{patch}")
            diff_snippet = truncate("\n\n".join(parts), 2000)

    # -----------------------------
    # Timeline snippet
    # -----------------------------
    
    timeline_snippet = ""
    if timeline is not None and "pr_id" in timeline.columns:
        tl_df = timeline[timeline["pr_id"] == pid].copy()
        if "created_at" in tl_df.columns:
            tl_df["created_at"] = pd.to_datetime(tl_df["created_at"], errors="coerce", utc=True)
            tl_df = tl_df.sort_values("created_at")
        last_events = tl_df.tail(20)
        lines = []
        for _, row in last_events.iterrows():
            ts    = safe_str(row.get("created_at"))
            ev    = safe_str(row.get("event"))
            actor = safe_str(row.get("actor_login") or row.get("actor"))
            details_parts = []
            for col in ["state", "label", "message"]:
                val = safe_str(row.get(col))
                if val and val.lower() not in ("nan", "none"):
                    details_parts.append(val)
            details = " | ".join(details_parts)
            lines.append(f"[{ts}] {actor} — {ev}: {details}")
        timeline_snippet = truncate("\n".join(lines), 2000)

    # -----------------------------
    # CI / workflow logs
    # -----------------------------
    
    ci_snippet = ""
    if timeline is not None and "pr_id" in timeline.columns:
        ci_df = timeline[timeline["pr_id"] == pid]
        ci_mask = ci_df["event"].astype(str).str.contains("fail|error|ci|workflow|build", case = False, na = False)
        ci_logs = []
        for _, row in ci_df.loc[ci_mask].iterrows():
            parts = []
            for field in ["event", "label", "message"]:
                val = safe_str(row.get(field))
                if val and val.lower() not in ("nan", "none"):
                    parts.append(val)
            joined = " | ".join(parts)
            if joined.strip():
                ci_logs.append(joined)
        ci_snippet = truncate("\n".join(ci_logs), 1500)

    # -----------------------------
    # Review + discussion text (reviews, review comments, PR comments)
    # -----------------------------
    
    review_texts = []

    # pr_reviews.body (review summaries)
    if reviews is not None and {"pr_id", "body"}.issubset(reviews.columns):
        sub = reviews[reviews["pr_id"] == pid]
        review_texts.extend(sub["body"].dropna().astype(str).tolist())

    # pr_review_comments.body (line-level review comments)
    if (rev_comments is not None
        and not rev_comments.empty
        and "pull_request_review_id" in rev_comments.columns
        and reviews is not None
        and "id" in reviews.columns
        and "pr_id" in reviews.columns):
        rev_ids = reviews.loc[reviews["pr_id"] == pid, "id"]
        sub2 = rev_comments[rev_comments["pull_request_review_id"].isin(rev_ids)]
        if "body" in sub2.columns:
            review_texts.extend(sub2["body"].dropna().astype(str).tolist())

    # general PR discussion comments
    if pr_comments is not None and {"pr_id", "body"}.issubset(pr_comments.columns):
        sub3 = pr_comments[pr_comments["pr_id"] == pid]
        review_texts.extend(sub3["body"].dropna().astype(str).tolist())

    # deduplicate while preserving order
    review_texts = list(dict.fromkeys(review_texts))
    review_snippet = truncate("\n---\n".join(review_texts), 2000)

    # -----------------------------
    # Commit messages (from pr_commits)
    # -----------------------------
    
    commit_snippet = ""
    if pr_commits is not None and {"pr_id", "message"}.issubset(pr_commits.columns):
        msgs = pr_commits[pr_commits["pr_id"] == pid]["message"].dropna().astype(str).tolist()
        commit_snippet = truncate("\n".join(msgs), 1000)

    # -----------------------------
    # Build the final context string
    # -----------------------------
    
    title = safe_str(pr.get("title", ""))
    body  = safe_str(pr.get("body", ""))

    ctx = f"""PR #{pid}
Repository: {repo_name} (id={repo_id}), language={repo_lang}
Author: {author_login} (author_type={author_type})
Outcome: {outcome} (state={state}, merged_at={merged_at})
Created at: {created_at}
Updated at: {updated_at}
Closed at: {closed_at}

Signals:
- ci_failed: {ci_failed_flag}
- changes_requested: {changes_requested_flag}

Title:
{title}

Description:
{truncate(body, 1000)}

=== DIFF SUMMARY ===
{diff_snippet or 'N/A'}

=== TIMELINE (last events) ===
{timeline_snippet or 'N/A'}

=== CI / WORKFLOW LOGS ===
{ci_snippet or 'N/A'}

=== REVIEW & DISCUSSION SNIPPET ===
{review_snippet or 'N/A'}

=== COMMIT MESSAGES ===
{commit_snippet or 'N/A'}
"""

    out_path = CONTEXT_DIR / f"PR_{pid}.txt"
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(ctx)

    contexts.append({"id": pid, "path": out_path.name})

print(f"✅ Saved {len(contexts)} context files to {CONTEXT_DIR}")

# save a small index of contexts for later use
contexts_df = pd.DataFrame(contexts)
contexts_df.to_csv(RESULTS_DIR / "failed_pr_context_index.csv", index=False)
contexts_df.head()

  5%|▍         | 369/7635 [00:01<00:34, 209.96it/s]


KeyboardInterrupt: 